In [1]:
import requests
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default='notebook_connected'

/Users/andreagurgone/Documents/bea_goods_services/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1. The pandemic has shifted consumers' preferences away from services and towards goods, in particular durable goods.
2. One way to look at it is to observe the supply curves for goods and services.
3. The Bureau of Economic Analysis (BEA) provides such a information in its NIPA tables (T20303 and T20304) on a quarterly basis for the United States
4. In what follows, I attempt to reproduce Figure 2 from 
Konczal, M., 2023. Inflation in 2023: Causes, Progress, and Solutions. https://rooseveltinstitute.org/wp-content/uploads/2023/03/RI_MikeKonczal_HouseTestimony_202303.pdf

### The first step is to set up the API to download data from BEA

In [ ]:
parameters = {
    "UserID": 'myAPIkey',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20303",  # T20304
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)
data = response.json()["BEAAPI"]["Results"]["Data"]

In [4]:
parameters = {
    "UserID": '9452074C-9BA9-4108-B38D-55B924516C1C',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20303",
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)

data = response.json()["BEAAPI"]["Results"]["Data"]
df_quantity = pd.DataFrame(data)
df_quantity["DataValue"] = pd.to_numeric(df_quantity["DataValue"])
df_quantity = df_quantity.pivot_table(index="TimePeriod", columns="LineDescription", values="DataValue")

parameters = {
    "UserID": '9452074C-9BA9-4108-B38D-55B924516C1C',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20304",
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)

data = response.json()["BEAAPI"]["Results"]["Data"]
df_price = pd.DataFrame(data)
df_price["DataValue"] = pd.to_numeric(df_price["DataValue"])
df_price = df_price.pivot_table(index="TimePeriod", columns="LineDescription", values="DataValue")

# Merge price and quantities into a dataframe
df_pq = df_quantity[['Goods', 'Services']]
df_pq = df_pq.rename(columns = {'Goods': 'Quantity (goods)', 'Services': 'Quantity (services)'})
df_pq['Price (goods)'] = df_price['Goods']
df_pq['Price (services)'] = df_price['Services']

latest_n_values = 20
df_pq_latest = df_pq[-latest_n_values:]
df_pq_latest.head(latest_n_values)

LineDescription,Quantity (goods),Quantity (services),Price (goods),Price (services)
TimePeriod,,,,
2019Q3,108.281,104.364,100.334,105.256
2019Q4,108.637,105.197,100.463,105.802
2020Q1,108.002,102.875,100.166,106.429
2020Q2,105.469,90.880,98.728,106.542
2020Q3,117.066,98.154,99.696,107.322
2020Q4,117.965,99.860,100.033,107.928
2021Q1,122.916,101.170,101.395,109.022
2021Q2,127.128,104.520,103.536,110.409
2021Q3,123.958,107.148,105.380,111.708


#### The content of the dataset in the last 10 periods is

In [18]:
df_pq.tail(10)

LineDescription,Quantity (goods),Quantity (services),Price (goods),Price (services)
TimePeriod,,,,
2020Q3,142.706,106.388,94.345,120.416
2020Q4,142.823,107.885,94.461,121.095
2021Q1,151.096,108.944,95.803,122.248
2021Q2,155.313,112.145,97.882,123.791
2021Q3,152.144,114.639,99.688,125.185
2021Q4,153.027,115.637,101.923,126.607
2022Q1,152.999,116.240,104.987,128.126
2022Q2,152.007,117.545,107.670,129.875
2022Q3,151.867,118.608,108.402,131.516


### Let's visualize the data in scatter plots

In [5]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Goods", "Services"),
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0.1,
                    specs=[[{"type": "scatter"}, {"type": "scatter"}]],
                    )

fig.add_trace(
    go.Scatter(x=df_pq_latest['Quantity (goods)'],
               y=df_pq_latest['Price (goods)'],
               mode='lines+markers+text',
               name='goods',
               # text=df_pq_latest.index,
               text=[index if i%2==0 else '' for i, index in enumerate(df_pq_latest.index)],
               textposition="top center",
               textfont=dict(size=10),
               marker=dict(size=8, line=dict(width=1, color="black")),
               showlegend=False),
    row=1, col=1
)

fig.update_xaxes(title_text="Quantity (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=1)
fig.update_yaxes(title_text="Price (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=1)

fig.add_trace(
    go.Scatter(x=df_pq_latest['Quantity (services)'],
               y=df_pq_latest['Price (services)'],
               mode='lines+markers+text',
               name='services',
            #    text=df_pq_latest.index,
               text=[index if i%2==0 else '' for i, index in enumerate(df_pq_latest.index)],
               textposition="top left",
               textfont=dict(size=10),
               marker=dict(size=8, line=dict(width=1, color="black")),
               showlegend=False),
    row=1, col=2
)

fig.update_xaxes(title_text="Quantity (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=2)
fig.update_yaxes(showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=2)

fig.update_layout(height=300, width=800,
                  margin=dict(l=50, r=50, t=50, b=50),
                  font=dict(family="Arial", size=12, color="black"),
                  plot_bgcolor='white',
                  )

In [6]:
fig.show()
# jupyter nbconvert BEA_pq_scatter.ipynb --to slides --post serve --template output_toggle --ExecutePreprocessor.enabled=True --ExecutePreprocessor.timeout=-1 --to=notebook
